In [10]:
# pip install cmake
# pip install face_recognition
# pip install opencv-python

In [11]:
# importing the necessary packages
import face_recognition
import cv2
import numpy as np
import csv
from datetime import datetime

In [12]:
video_capture = cv2.VideoCapture(0)

In [13]:
# Load known faces
nawaz_image = face_recognition.load_image_file("faces/1.png")
#Encoding the image
# Encoding: Connverting image to numbers
nawaz_face_encoding = face_recognition.face_encodings(nawaz_image)[0]

babloo_image = face_recognition.load_image_file("faces/babloo.jpg")
babloo_face_encoding = face_recognition.face_encodings(babloo_image)[0]

vaishnavi_image = face_recognition.load_image_file("faces/vaishnavi_singh.jpeg")
vaishnavi_image_encoding = face_recognition.face_encodings(vaishnavi_image)[0]

In [14]:
known_face_encodings = [nawaz_face_encoding,babloo_face_encoding,vaishnavi_image_encoding]
known_face_names = ["Nawaz","Babloo","Vaishnavi"]

In [15]:
# List of Students
student =known_face_names.copy()

face_locations = []
face_encodings = []

In [16]:
# Get current date and time
now = datetime.now()
current_date = now.strftime("%d-%m-%Y")


In [17]:
# Opening the csv file in append mode with file pointer as fp
f = open(f"{current_date}.csv", "w+", newline="")
writer = csv.writer(f)

In [18]:
while True:
    _, frame = video_capture.read()
    small_frame = cv2.resize(frame,(0,0),fx=0.75,fy=0.75)
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    # Recognize faces
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    name = None
    
    for face in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings,face)
        face_distance = face_recognition.face_distance(known_face_encodings,face)
        best_match_index = np.argmin(face_distance)

        if matches[best_match_index]:
            name = known_face_names[best_match_index]
    
        # Add text if a person is present
        if name in known_face_names:
            font = cv2.FONT_HERSHEY_SIMPLEX
            bottomLeftCornerOfText = (10,50)
            fontScale = 1.5
            fontColor = (0,0,0)
            lineType = 2
            thickness = 3
            cv2.putText(frame,f"Present: {name}",bottomLeftCornerOfText,font,fontScale,fontColor,thickness,lineType)


            if name in student:
                student.remove(name)
                current_time = now.strftime("%H:%M:%S")
                writer.writerow([name,current_time])

    cv2.imshow("Attendance",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()
f.close()         